# [Semi-Quantum Key Distribution](https://arxiv.org/pdf/0812.4835.pdf)

In [148]:
from qiskit import *
import math
import numpy as np
simulator = Aer.get_backend('qasm_simulator')

## Protocol 2

In [149]:
def encode(basis, val):
	'''
	basis = 0 (Computational), 1 (Hadamard)
	'''
	circuit = QuantumCircuit(1, 1)
	if val == "1":
		circuit.x(0)
	if basis == 1:
		circuit.h(0)
	return circuit

In [150]:
delta = 1/8
n = 2
N = math.ceil(8*n*(1+delta))
N

18

In [151]:
message = "".join([str(i) for i in list(np.random.randint(2, size=N))])
alice_basis = np.random.randint(2, size=N)
bob_basis = np.random.randint(2, size=N)
message, alice_basis, bob_basis

('000110011001011000',
 array([0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]),
 array([0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]))

In [152]:
circuits = [None]*N

In [153]:
for i in range(N):
    circuits[i] = encode(alice_basis[i], message[i])

In [154]:
results_bob = [None]*len(circuits)
counts_bob = [None]*len(circuits)

for i in range(len(circuits)):
    if bob_basis[i] == 1:
        circuits[i].measure(0, 0)
        results_bob[i] = execute(circuits[i], backend=simulator).result()
        counts_bob[i] = results_bob[i].get_counts(circuits[i])

counts_bob

[None,
 {'0': 1024},
 {'1': 502, '0': 522},
 None,
 {'1': 1024},
 {'0': 1024},
 None,
 {'0': 529, '1': 495},
 None,
 {'0': 1024},
 {'0': 1024},
 {'1': 1024},
 {'0': 536, '1': 488},
 {'1': 1024},
 {'1': 1024},
 None,
 None,
 {'0': 1024}]

In [155]:
for i in range(N):
    if bob_basis[i] == 0:
        if alice_basis[i] == 1:
            circuits[i].h(0)
        circuits[i].measure(0, 0)

In [156]:
results_ctrl = [None]*len(circuits)
counts_ctrl = [None]*len(circuits)

for i in range(len(circuits)):
    if bob_basis[i] == 0:
        results_ctrl[i] = execute(circuits[i], backend=simulator).result()
        counts_ctrl[i] = results_ctrl[i].get_counts(circuits[i])
counts_ctrl

[{'0': 1024},
 None,
 None,
 {'1': 1024},
 None,
 None,
 {'0': 1024},
 None,
 {'1': 1024},
 None,
 None,
 None,
 None,
 None,
 None,
 {'0': 1024},
 {'0': 1024},
 None]

In [157]:
P_ctrl = 0

for i in range(len(message)):
    if counts_ctrl[i] and list(counts_ctrl[i].keys())[0] != message[i]:
        P_ctrl += 1
P_ctrl

0

In [158]:
results_sift = [None]*len(circuits)
counts_sift = [None]*len(circuits)

for i in range(len(circuits)):
    if bob_basis[i] == 1:
        results_sift[i] = execute(circuits[i], backend=simulator).result()
        counts_sift[i] = results_sift[i].get_counts(circuits[i])
counts_sift

[None,
 {'0': 1024},
 {'1': 513, '0': 511},
 None,
 {'1': 1024},
 {'0': 1024},
 None,
 {'0': 502, '1': 522},
 None,
 {'0': 1024},
 {'0': 1024},
 {'1': 1024},
 {'0': 510, '1': 514},
 {'1': 1024},
 {'1': 1024},
 None,
 None,
 {'0': 1024}]

In [159]:
P_sift = 0

for i in range(len(message)):
    if counts_sift[i] and list(counts_sift[i].keys())[0] != message[i]:
        P_sift += 1
P_sift

2

Instead of P_sift on all qubits, some sampled qubits are used. If error rate is below threshold. Then the remaining sift bits are the message